In [1]:
!pip install numpy tensorflow opencv-python streamlit streamlit-drawable-canvas

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize values between 0 and 1
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [7]:

# Build the CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [9]:
# Train the model
model = create_model()
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))



C:\Users\aryai\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.9164 - loss: 0.2763 - val_accuracy: 0.9817 - val_loss: 0.0495
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9865 - loss: 0.0424 - val_accuracy: 0.9884 - val_loss: 0.0366
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9914 - loss: 0.0277 - val_accuracy: 0.9893 - val_loss: 0.0321
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9931 - loss: 0.0193 - val_accuracy: 0.9907 - val_loss: 0.0270
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9961 - loss: 0.0120 - val_accuracy: 0.9916 - val_loss: 0.0266


In [10]:
# Save the trained model
model.save("digit_model.h5")

print("Model training complete and saved as 'digit_model.h5'")


Model training complete and saved as 'digit_model.h5'


In [13]:
with open("app.py", "w") as f:
    f.write("""
import numpy as np
import tensorflow as tf
import cv2
import streamlit as st
from streamlit_drawable_canvas import st_canvas

# Load the trained model
model = tf.keras.models.load_model("digit_model.h5")

# Streamlit UI
def main():
    st.title("Digit Recognition")
    st.write("Draw a digit (0-9) below:")

    # Create a drawing canvas
    canvas_result = st_canvas(
        fill_color="black",  # Background color
        stroke_width=10,
        stroke_color="white",
        background_color="black",
        height=280,
        width=280,
        drawing_mode="freedraw",
        key="canvas"
    )

    if canvas_result.image_data is not None:
        img = canvas_result.image_data
        img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGBA2GRAY)
        img = cv2.resize(img, (28,28))
        img = img / 255.0  # Normalize
        img = img.reshape(1, 28, 28, 1)
        
        prediction = model.predict(img)
        st.write("Predicted Digit:", np.argmax(prediction))

if __name__ == "__main__":
    main()
    """)


In [ ]:
!streamlit run app.py
